In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import opendatasets as od
import tensorflow as tf
import cv2
from tqdm import *

In [2]:
dataset_directory = "C:\\Users\\amith\\Documents\\Datasets\\kaggle\\kaggle".replace("\\","/")
csv_directory = "C:\\Users\\amith\\Documents\\Datasets".replace("\\","/")

In [3]:
train_val_age = pd.read_csv(csv_directory + "/train_age.csv").sort_values(by="imageId")
train_val_gender = pd.read_csv(csv_directory + "/train_gender.csv").sort_values(by="imageId")

id = train_val_age["imageId"].values.tolist()
train_val_files = pd.DataFrame(columns=["imageId","Files"])
ctr = 0

for i in os.listdir(dataset_directory+"/train"):
    train_val_files.loc[ctr,"imageId"] = id[ctr]
    train_val_files.loc[ctr,"Files"] = i
    ctr += 1

train_val_files = train_val_files.sort_values(by="imageId")

train_val_output = pd.merge(left=train_val_age,right=train_val_gender,on="imageId",how="inner")
train_val_output = pd.merge(left=train_val_output,right=train_val_files,on="imageId",how="inner")

train_val_output = train_val_output.sort_values(by="imageId")

train_val_id = train_val_output["imageId"].values.tolist()
np.random.shuffle(train_val_id)


threshold = int(np.round(0.8 * len(train_val_id)))

train_id = train_val_id[0:threshold]
val_id = train_val_id[threshold:]

train_val_output.set_index(keys="imageId",drop=False,inplace=True)

train_output = train_val_output.loc[train_id,:]
val_output = train_val_output.loc[val_id,:]

train_output = train_output.reset_index(drop=True).drop(columns=["imageId"])
val_output = val_output.reset_index(drop=True).drop(columns=["imageId"])

In [4]:
train_output.head()

,age,gender,Files
0,85.0,0,006419.png
1,42.0,1,005273.png
2,47.0,0,005102.png
3,78.0,1,003927.png
4,67.0,0,010516.png


In [5]:
val_output.head()

,age,gender,Files
0,76.0,1,000209.png
1,66.0,0,004918.png
2,64.0,1,000900.png
3,73.0,0,003272.png
4,35.0,0,002047.png


In [6]:
train_images = []
train_data = {}
ctr = 0
for i in tqdm(iterable=train_output["Files"],desc="Processing train data"):
    ages = train_output.loc[ctr,"age"]
    genders = train_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    train_images.append(img)
    ctr += 1

train_images = np.array(train_images)

Processing train data: 100%|██████████| 8562/8562 [02:39<00:00, 53.70it/s]


In [7]:
dataset_directory + "/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/001657.png'

In [8]:
dataset_directory + "/val/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/val/001657.png'

In [9]:
val_images = []
val_data = {}
ctr = 0
for i in tqdm(iterable=val_output["Files"],desc="Processing val data"):
    ages = val_output.loc[ctr,"age"]
    genders = val_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    val_images.append(img)
    ctr += 1

val_images = np.array(val_images)

Processing val data: 100%|██████████| 2140/2140 [00:40<00:00, 53.15it/s]


In [10]:
train_images = train_images.reshape(tuple(list(train_images.shape) + [1]))
val_images = val_images.reshape(tuple(list(val_images.shape) + [1]))

In [11]:
val_images.shape

(2140, 64, 64, 1)

In [12]:
model_genders = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="softmax")
])

In [13]:
model_ages = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="relu")
])

In [14]:
train_ages = train_output["age"].values
train_genders = train_output["gender"].values

val_ages = val_output["age"].values
val_genders = val_output["gender"].values

In [15]:
model_genders.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy'])
model_ages.compile(optimizer=tf.keras.optimizers.SGD(),loss=tf.keras.losses.mean_absolute_error)

In [16]:
model_genders.fit(x=train_images,y=train_genders,batch_size=32,epochs=100,validation_data=(val_images,val_genders),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/100
268/268 [==============================] - 112s 414ms/step - loss: 0.4401 - accuracy: 0.4202 - val_loss: 0.3453 - val_accuracy: 0.4182
Epoch 2/100
268/268 [==============================] - 113s 423ms/step - loss: 0.2895 - accuracy: 0.4202 - val_loss: 0.2770 - val_accuracy: 0.4182
Epoch 3/100
268/268 [==============================] - 39s 145ms/step - loss: 0.2541 - accuracy: 0.4202 - val_loss: 0.2521 - val_accuracy: 0.4182
Epoch 4/100
268/268 [==============================] - 40s 149ms/step - loss: 0.1922 - accuracy: 0.4202 - val_loss: 0.2496 - val_accuracy: 0.4182
Epoch 5/100
268/268 [==============================] - 41s 151ms/step - loss: 0.1536 - accuracy: 0.4202 - val_loss: 0.2107 - val_accuracy: 0.4182
Epoch 6/100
268/268 [==============================] - 41s 153ms/step - loss: 0.1176 - accuracy: 0.4202 - val_loss: 0.2428 - val_accuracy: 0.4182
Epoch 7/100
268/268 [==============================] - 41s 155ms/step - loss: 0.0898 - accuracy: 0.4202 - val_loss: 0.2325

In [17]:
model_ages.fit(x=train_images,y=train_ages,batch_size=32,epochs=100,validation_data=(val_images,val_ages),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/100
268/268 [==============================] - 39s 141ms/step - loss: 26.1029 - val_loss: 49.0051
Epoch 2/100
268/268 [==============================] - 37s 139ms/step - loss: 23.2380 - val_loss: 17.6199
Epoch 3/100
268/268 [==============================] - 38s 141ms/step - loss: 22.2441 - val_loss: 24.5099
Epoch 4/100
268/268 [==============================] - 38s 142ms/step - loss: 21.6314 - val_loss: 20.1874


In [18]:
tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)

C:\Users\amith\AppData\Local\Temp\ipykernel_40032\4190216403.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
C:\Users\amith\AppData\Local\Temp\ipykernel_40032\4190216403.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)
